# Prepare database

In [4]:
import os

dir_name = 'C:/Users/windo/Documents/ravdess-dataset/audio_speech_actors_01-24'

# Initialize a list to hold the file names
all_files = []

# Traverse all subfolders and files using os.walk()
for root, _, file_names in os.walk(dir_name):
    for file_name in file_names:
        file_path = os.path.join(root, file_name)
        if os.path.isfile(file_path):  # Check if it's a file
            all_files.append(file_path)

all_files = sorted(all_files)

# # Display the collected file names
# print('List of all files in the folders:')
# for file in all_files:
#     print(file)

# Emotions in the RAVDESS dataset
emotions = {
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

# desired csv columns:
# label/name(actor number, take, etc) emotion(class) feature1 feature2...
# or class at the end?
len(all_files)

1440

In [7]:
import numpy as np

import sys
sys.path.append("..")
import plap

# Use plap for parameterization
fvector = plap.FeatureVector("asc","ass","asf","lat","tc","sc","hsc","hsd","hss","hsv")
# preprocessor = plap.Preprocessor(window_type="hamming") # !!! THIS HAD THE DEFAULT PRE-EMPHASIS XDDXDDDDD
preprocessor = plap.Preprocessor(preemphasis_coeff=None, window_type="hamming")

def extract_features(audio_path):
    plap.parameterize(audio_path=audio_path, fvector=fvector, preprocessor=preprocessor)
    return fvector.values

In [ ]:
# import csv

# # Define the CSV filename
# csv_filename = './python_csv/ravdess_basspectr_timbral_no_preemphasis.csv'

# # Initialize the CSV file with headers
# headers = ['label', 'class'] + [f'{feature.upper()}' for feature in fvector.features]
# with open(csv_filename, mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(headers)

In [ ]:
import time
# Iterate over each audio file
initial_i = 150
# end_i = 150
i = initial_i
processed = 0
for filename in all_files[initial_i:]:
    if filename.endswith('.wav'):  # Ensure only audio files are processed
        
        start_time = time.time()
        # Extract label and class from the filename (customize this logic as needed)
        audio_filename = filename.split('\\')[-1]
        label = audio_filename.removesuffix('.wav')  # Use filename as label
        class_num = audio_filename.split('-')[2]
        class_name = emotions[class_num]
        
        print(f'Processing file {audio_filename}')

        # Extract 11 features from the audio file
        try:
            features = extract_features(filename)
            processed += 1
        except:
            print(f'Omitting file{i}')
            i += 1
            continue
        # features = np.zeros(11)

        # Prepare the row to be added to the CSV
        row = [label, class_name] + list(features)

        # Append the new row to the CSV file
        with open(csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(row)
        print(f'Finished file{i} in {time.time() - start_time} s')
        i += 1

print("Feature extraction and CSV writing completed.")

Processing file 03-01-05-01-02-01-03.wav
Finished file150 in 4.111996412277222 s
Processing file 03-01-05-01-02-02-03.wav
Finished file151 in 4.364858865737915 s
Processing file 03-01-05-02-01-01-03.wav
Finished file152 in 4.6375486850738525 s
Processing file 03-01-05-02-01-02-03.wav
Finished file153 in 4.71860671043396 s
Processing file 03-01-05-02-02-01-03.wav
Finished file154 in 5.021414756774902 s
Processing file 03-01-05-02-02-02-03.wav
Finished file155 in 5.099766254425049 s
Processing file 03-01-06-01-01-01-03.wav
Finished file156 in 4.782593727111816 s
Processing file 03-01-06-01-01-02-03.wav
Finished file157 in 5.258265256881714 s
Processing file 03-01-06-01-02-01-03.wav
Finished file158 in 4.387857913970947 s
Processing file 03-01-06-01-02-02-03.wav
Finished file159 in 3.8505003452301025 s
Processing file 03-01-06-02-01-01-03.wav
Finished file160 in 3.655369997024536 s
Processing file 03-01-06-02-01-02-03.wav
Finished file161 in 4.123462200164795 s
Processing file 03-01-06-02

c:\Users\windo\Desktop\Inżynierka\plap\thesis\..\plap\parameterization\mpeg7.py:743: RuntimeWarning: divide by zero encountered in scalar divide
  HarmonicSpectralVariation = 1 - crossprod / (np.sqrt(temp))


Finished file364 in 5.529875040054321 s
Processing file 03-01-02-01-01-02-07.wav
Finished file365 in 3.9344065189361572 s
Processing file 03-01-02-01-02-01-07.wav
Finished file366 in 4.897547483444214 s
Processing file 03-01-02-01-02-02-07.wav
Finished file367 in 4.217513561248779 s
Processing file 03-01-02-02-01-01-07.wav
Finished file368 in 4.2337117195129395 s
Processing file 03-01-02-02-01-02-07.wav
Finished file369 in 3.9991323947906494 s
Processing file 03-01-02-02-02-01-07.wav
Finished file370 in 4.349617958068848 s
Processing file 03-01-02-02-02-02-07.wav
Finished file371 in 4.115302801132202 s
Processing file 03-01-03-01-01-01-07.wav
Finished file372 in 4.58418607711792 s
Processing file 03-01-03-01-01-02-07.wav
Finished file373 in 3.6960411071777344 s
Processing file 03-01-03-01-02-01-07.wav
Finished file374 in 3.6956286430358887 s
Processing file 03-01-03-01-02-02-07.wav
Finished file375 in 4.31413459777832 s
Processing file 03-01-03-02-01-01-07.wav
Finished file376 in 4.684